In [4]:
import pandas as pd
import numpy as np

In [19]:
tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN']
date_range = pd.date_range('2024-01-01', periods=3, freq='D')
columns = ['Predicted_Model1', 'Predicted_Model2']

# Create a multi-index with 'Ticker' and 'Date'
rows = pd.MultiIndex.from_product([tickers, date_range], names=['Ticker', 'Date'])

# Create the DataFrame with random values in the specified range
df_trading = pd.DataFrame(data=np.random.uniform(-1, 1, size=(len(rows), len(columns))),
                          index=rows,
                          columns=columns)

df_trading

Predicted_Model1  Predicted_Model2
Ticker Date                                          
AAPL   2024-01-01          0.707215          0.104136
       2024-01-02         -0.268853          0.965832
       2024-01-03         -0.043445          0.729304
GOOGL  2024-01-01         -0.099312         -0.650936
       2024-01-02          0.129564          0.255234
       2024-01-03          0.269593         -0.878427
MSFT   2024-01-01         -0.915049          0.834031
       2024-01-02          0.349339         -0.829665
       2024-01-03          0.769553         -0.119333
TSLA   2024-01-01         -0.728246         -0.629871
       2024-01-02          0.285134         -0.996741
       2024-01-03          0.743554          0.794996
AMZN   2024-01-01         -0.651291          0.718894
       2024-01-02         -0.232508          0.796867
       2024-01-03          0.407785         -0.853931

In [26]:
df_signal = ( df_trading > 0 ) # 1: Buy, 0: Sell
df_signal = df_signal.rename(columns={'Predicted_Model1': 'Signal_model1',
                                      'Predicted_Model2': 'Signal_model2'})
df_signal

Signal_model1  Signal_model2
Ticker Date                                    
AAPL   2024-01-01           True           True
       2024-01-02          False           True
       2024-01-03          False           True
GOOGL  2024-01-01          False          False
       2024-01-02           True           True
       2024-01-03           True          False
MSFT   2024-01-01          False           True
       2024-01-02           True          False
       2024-01-03           True          False
TSLA   2024-01-01          False          False
       2024-01-02           True          False
       2024-01-03           True           True
AMZN   2024-01-01          False           True
       2024-01-02          False           True
       2024-01-03           True          False

In [31]:
df_trading = df_trading.join(df_signal)

In [36]:
df_trading['Trade_Return_Model1'] = df_trading['Predicted_Model1'] * df_trading['Signal_model1']
df_trading['Trade_Return_Model2'] = df_trading['Predicted_Model2'] * df_trading['Signal_model2']

In [37]:
df_trading

Predicted_Model1  Predicted_Model2  Signal_model1  \
Ticker Date                                                            
AAPL   2024-01-01          0.707215          0.104136           True   
       2024-01-02         -0.268853          0.965832          False   
       2024-01-03         -0.043445          0.729304          False   
GOOGL  2024-01-01         -0.099312         -0.650936          False   
       2024-01-02          0.129564          0.255234           True   
       2024-01-03          0.269593         -0.878427           True   
MSFT   2024-01-01         -0.915049          0.834031          False   
       2024-01-02          0.349339         -0.829665           True   
       2024-01-03          0.769553         -0.119333           True   
TSLA   2024-01-01         -0.728246         -0.629871          False   
       2024-01-02          0.285134         -0.996741           True   
       2024-01-03          0.743554          0.794996           True   
AMZN   2024-01-01         -0.651291          0.718894          False   
       2024-01-02         -0.232508          0.796867          False   
       2024-01-03          0.407785         -0.853931           True   

                   Signal_model2  Trade_Return_Model1  Trade_Return_Model2  
Ticker Date                                                                 
AAPL   2024-01-01           True             0.707215             0.104136  
       2024-01-02           True            -0.000000             0.965832  
       2024-01-03           True            -0.000000             0.729304  
GOOGL  2024-01-01          False            -0.000000            -0.000000  
       2024-01-02           True             0.129564             0.255234  
       2024-01-03          False             0.269593            -0.000000  
MSFT   2024-01-01           True            -0.000000             0.834031  
       2024-01-02          False             0.349339            -0.000000  
       2024-01-03          False             0.769553            -0.000000  
TSLA   2024-01-01          False            -0.000000            -0.000000  
       2024-01-02          False             0.285134            -0.000000  
       2024-01-03           True             0.743554             0.794996  
AMZN   2024-01-01           True            -0.000000             0.718894  
       2024-01-02           True            -0.000000             0.796867  
       2024-01-03          False             0.407785            -0.000000

In [38]:
df_trading.sum()

Predicted_Model1       0.723034
Predicted_Model2       0.240390
Signal_model1          8.000000
Signal_model2          8.000000
Trade_Return_Model1    3.661737
Trade_Return_Model2    5.199294
dtype: float64

In [43]:
# Assume a risk-free rate of 0% for simplicity
risk_free_rate = 0.0

# Reset index to make 'Date' a column
df_trading_reset = df_trading.reset_index()

# Calculate daily returns
df_trading_reset['Daily_Return_Model1'] = df_trading_reset.groupby('Date')['Trade_Return_Model1'].transform('sum')
df_trading_reset['Daily_Return_Model2'] = df_trading_reset.groupby('Date')['Trade_Return_Model2'].transform('sum')

# Calculate mean daily return and standard deviation of daily return
mean_daily_return_model1 = df_trading_reset['Daily_Return_Model1'].mean()
mean_daily_return_model2 = df_trading_reset['Daily_Return_Model2'].mean()
std_daily_return_model1 = df_trading_reset['Daily_Return_Model1'].std()
std_daily_return_model2 = df_trading_reset['Daily_Return_Model2'].std()

# Calculate Sharpe Ratio
sharpe_ratio_model1 = (mean_daily_return_model1 - risk_free_rate) / std_daily_return_model1
sharpe_ratio_model2 = (mean_daily_return_model2 - risk_free_rate) / std_daily_return_model2

sharpe_ratio_model1, sharpe_ratio_model2

(1.718389799912238, 8.027533447205094)

In [44]:
# Define portfolio weights (e.g., equal weights for each asset)
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Calculate daily portfolio return
df_trading_reset['Daily_Portfolio_Return'] = (df_trading_reset['Trade_Return_Model1'] * weights[0] +
                                              df_trading_reset['Trade_Return_Model2'] * weights[1])

# Calculate mean daily return and standard deviation of daily return for the portfolio
mean_daily_return_portfolio = df_trading_reset['Daily_Portfolio_Return'].mean()
std_daily_return_portfolio = df_trading_reset['Daily_Portfolio_Return'].std()

# Calculate Sharpe Ratio for the portfolio
sharpe_ratio_portfolio = (mean_daily_return_portfolio - risk_free_rate) / std_daily_return_portfolio

# Perform portfolio optimization (e.g., using scipy's minimize function)
# Note: This is a simple example and may need to be adapted based on your specific optimization criteria and constraints
from scipy.optimize import minimize

def objective(weights):
    return -((df_trading_reset['Trade_Return_Model1'] * weights[0] +
               df_trading_reset['Trade_Return_Model2'] * weights[1]).mean())

initial_guess = np.array([0.5, 0.5])  # Initial guess for weights
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Constraint: weights sum to 1
optimized_weights = minimize(objective, initial_guess, constraints=constraints)['x']

# Calculate optimized portfolio return and Sharpe Ratio
optimized_portfolio_return = (df_trading_reset['Trade_Return_Model1'] * optimized_weights[0] +
                              df_trading_reset['Trade_Return_Model2'] * optimized_weights[1]).mean()
optimized_sharpe_ratio = (optimized_portfolio_return - risk_free_rate) / (df_trading_reset['Trade_Return_Model1'] * optimized_weights[0] +
                                                                         df_trading_reset['Trade_Return_Model2'] * optimized_weights[1]).std()

sharpe_ratio_portfolio, optimized_weights, optimized_sharpe_ratio


(1.4620255418520944,
 array([-9.1645477e+13,  9.1645477e+13]),
 0.1793863743596847)

In [45]:
# Define the number of models
num_models = 2

# Define the weights for each model
weights_per_model = np.ones((num_models, len(tickers))) / len(tickers)

# Calculate the daily portfolio return for each model
for i in range(num_models):
    df_trading_reset[f'Daily_Portfolio_Return_Model{i + 1}'] = np.sum(df_trading_reset[[f'Trade_Return_Model{i + 1}' for i in range(num_models)]] * weights_per_model[i], axis=1)

# Calculate mean daily return and standard deviation of daily return for each model's portfolio
mean_daily_return_per_model = [df_trading_reset[f'Daily_Portfolio_Return_Model{i + 1}'].mean() for i in range(num_models)]
std_daily_return_per_model = [df_trading_reset[f'Daily_Portfolio_Return_Model{i + 1}'].std() for i in range(num_models)]

# Calculate Sharpe Ratio for each model's portfolio
sharpe_ratio_per_model = [(mean_daily_return_per_model[i] - risk_free_rate) / std_daily_return_per_model[i] for i in range(num_models)]

sharpe_ratio_per_model


ValueError: Unable to coerce to Series, length must be 2: given 5